### Writing the multilayer perceptron class

In [2]:
import numpy as np

In [6]:
class Perceptron:
    """A single neuron with the sigmoid activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By default it's 1.0."""

    def __init__(self, inputs, bias=1.0):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias)."""
        self.weights = (np.random.rand(inputs+1) * 2) - 1
        self.bias = bias

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        x_sum = np.dot(np.append(x, self.bias), self.weights)
        return self.sigmoid(x_sum)

    def set_weights(self, w_init):
        """Set the weights. w_init is a python list with the weights."""
        self.weights = np.array(w_init)

    def sigmoid(self, x):
        """Evaluate the sigmoid function for the floating point input x."""
        return 1/(1+np.exp(-x))


class MultiLayerPerceptron:
    """A multilayer perceptron class that uses the Perceptron class above.
       Attributes:
          layers:  A python list with the number of elements per layer.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate."""

    def __init__(self, layers, bias=1.0, eta=0.5):
        """Return a new MLP object with the specified parameters."""
        self.layers = np.array(layers, dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = []  # The list of lists of neurons
        self.values = []  # The list of lists of output values
        self.d = []       # The list of lists of error terms (lowercase deltas)

        for i in range(len(self.layers)):
            self.values.append([])
            self.d.append([])
            self.network.append([])
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d[i] = [0.0 for j in range(self.layers[i])]
            if i > 0:  # network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]):
                    self.network[i].append(Perceptron(
                        inputs=self.layers[i-1], bias=self.bias))

        self.network = np.array([np.array(x)
                                for x in self.network], dtype=object)
        self.values = np.array([np.array(x)
                               for x in self.values], dtype=object)
        self.d = np.array([np.array(x) for x in self.d], dtype=object)

    def set_weights(self, w_init):
        """Set the weights. 
           w_init is a 3D list with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j])

    def print_weights(self):
        print()
        for i in range(1, len(self.network)):
            for j in range(self.layers[i]):
                print("Layer", i+1, "Neuron", j, self.network[i][j].weights)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Perceptron."""
        x = np.array(x, dtype=object)
        self.values[0] = x
        for i in range(1, len(self.network)):
            for j in range(self.layers[i]):
                self.values[i][j] = self.network[i][j].run(self.values[i-1])
        return self.values[-1]

    def bp(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm."""
        x = np.array(x, dtype=object)
        y = np.array(y, dtype=object)

        # Backpropagation Step by Step:

        # STEP 1: Feed a sample to the network
        outputs = self.run(x)

        # STEP 2: Calculate the MSE
        error = (y - outputs)
        MSE = sum(error ** 2) / self.layers[-1]

        # STEP 3: Calculate the output error terms
        self.d[-1] = outputs * (1 - outputs) * (error)

        # STEP 4: Calculate the error term of each unit on each layer
        for i in reversed(range(1, len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]):
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error

        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1, len(self.network)):
            for j in range(self.layers[i]):
                for k in range(self.layers[i-1]+1):
                    if k == self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k]
                    self.network[i][j].weights[k] += delta
        return MSE


# test code
mlp = MultiLayerPerceptron(layers=[2, 2, 1])
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(3000):
    mse = 0.0
    mse += mlp.bp([0, 0], [0])
    mse += mlp.bp([0, 1], [1])
    mse += mlp.bp([1, 0], [1])
    mse += mlp.bp([1, 1], [0])
    mse = mse / 4
    if (i % 100 == 0):
        print(mse)

mlp.print_weights()

print("MLP:")
print("0 0 = {0:.10f}".format(mlp.run([0, 0])[0]))
print("0 1 = {0:.10f}".format(mlp.run([0, 1])[0]))
print("1 0 = {0:.10f}".format(mlp.run([1, 0])[0]))
print("1 1 = {0:.10f}".format(mlp.run([1, 1])[0]))



Training Neural Network as an XOR Gate...

0.2754118120217931
0.25640119459857064
0.24825203849967992
0.22900344563001238
0.20114977880322105
0.17958199560136257
0.16415695328503482
0.1537633350118473
0.14727516477678046
0.14315312130961197
0.1403946196247723
0.13845234215684088
0.13702533811690676
0.1359400500974187
0.13509111907465027
0.13441154133907568
0.13385695217516058
0.1333969435285084
0.1330100463336673
0.13268071055387398
0.13239742093154172
0.13215148335372146
0.13193622087098655
0.1317464276548734
0.1315779898224999
0.1314276168610628
0.13129264797281376
0.1311709101784737
0.1310606128196562
0.13096026807617733

Layer 2 Neuron 0 [-7.86722247 -5.14831487  1.11966177]
Layer 2 Neuron 1 [-6.56989486  1.26218126 -0.59041099]
Layer 3 Neuron 0 [-6.77153673  4.85288022 -0.02919154]

MLP:
0 0 = 0.0321650744
0 1 = 0.9554049741
1 0 = 0.4916599328
1 1 = 0.4960113321


In [ ]:
import numpy as np

class Perceptron:
    """A single neuron with the sigmoid activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By default it's 1.0."""

    def __init__(self, inputs, bias = 1.0):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias).""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        x_sum = np.dot(np.append(x,self.bias),self.weights)
        return self.sigmoid(x_sum)

    def set_weights(self, w_init):
        """Set the weights. w_init is a python list with the weights."""
        self.weights = np.array(w_init)

    def sigmoid(self, x):
        """Evaluate the sigmoid function for the floating point input x."""
        return 1/(1+np.exp(-x))



class MultiLayerPerceptron:     
    """A multilayer perceptron class that uses the Perceptron class above.
       Attributes:
          layers:  A python list with the number of elements per layer.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate."""

    def __init__(self, layers, bias = 1.0, eta = 0.5):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons
        self.values = []  # The list of lists of output values
        self.d = []       # The list of lists of error terms (lowercase deltas)

        for i in range(len(self.layers)):
            self.values.append([])
            self.d.append([])
            self.network.append([])
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d[i] = [0.0 for j in range(self.layers[i])]
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): 
                    self.network[i].append(Perceptron(inputs = self.layers[i-1], bias = self.bias))
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object)
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init):
        """Set the weights. 
           w_init is a 3D list with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j])

    def print_weights(self):
        print()
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i+1,"Neuron",j,self.network[i][j].weights)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Perceptron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1])
        return self.values[-1]
    
    def bp(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)

        # Backpropagation Step by Step:

        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        
        # STEP 2: Calculate the MSE
        error = (y - outputs)
        MSE = sum( error ** 2) / self.layers[-1]

        # STEP 3: Calculate the output error terms
        self.d[-1] = outputs * (1 - outputs) * (error)

        # STEP 4: Calculate the error term of each unit on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error

        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                for k in range(self.layers[i-1]+1):
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k]
                    self.network[i][j].weights[k] += delta
        return MSE



#test code
epochs = int(input("How many epochs? "))
mlp1 = MultiLayerPerceptron(layers=[7,7,1])
mlp2 = MultiLayerPerceptron(layers=[7,7,10])
mlp3 = MultiLayerPerceptron(layers=[7,7,7])

# Dataset for the 7 to 1 network
print("Training 7 to 1 network...")
for i in range(epochs):
    mse = 0.0
    mse += mlp1.bp([1,1,1,1,1,1,0],[0.05])    #0 pattern
    mse += mlp1.bp([0,1,1,0,0,0,0],[0.15])    #1 pattern
    mse += mlp1.bp([1,1,0,1,1,0,1],[0.25])    #2 pattern
    mse += mlp1.bp([1,1,1,1,0,0,1],[0.35])    #3 pattern
    mse += mlp1.bp([0,1,1,0,0,1,1],[0.45])    #4 pattern
    mse += mlp1.bp([1,0,1,1,0,1,1],[0.55])    #5 pattern
    mse += mlp1.bp([1,0,1,1,1,1,1],[0.65])    #6 pattern
    mse += mlp1.bp([1,1,1,0,0,0,0],[0.75])    #7 pattern
    mse += mlp1.bp([1,1,1,1,1,1,1],[0.85])    #8 pattern
    mse += mlp1.bp([1,1,1,1,0,1,1],[0.95])    #9 pattern
    mse = mse/10.0

# Dataset for the 7 to 10 network
print("Training 7 to 10 network...")
for i in range(epochs):
    mse = 0.0
    mse += mlp2.bp([1,1,1,1,1,1,0],[1,0,0,0,0,0,0,0,0,0])    #0 pattern
    mse += mlp2.bp([0,1,1,0,0,0,0],[0,1,0,0,0,0,0,0,0,0])    #1 pattern
    mse += mlp2.bp([1,1,0,1,1,0,1],[0,0,1,0,0,0,0,0,0,0])    #2 pattern
    mse += mlp2.bp([1,1,1,1,0,0,1],[0,0,0,1,0,0,0,0,0,0])    #3 pattern
    mse += mlp2.bp([0,1,1,0,0,1,1],[0,0,0,0,1,0,0,0,0,0])    #4 pattern
    mse += mlp2.bp([1,0,1,1,0,1,1],[0,0,0,0,0,1,0,0,0,0])    #5 pattern
    mse += mlp2.bp([1,0,1,1,1,1,1],[0,0,0,0,0,0,1,0,0,0])    #6 pattern
    mse += mlp2.bp([1,1,1,0,0,0,0],[0,0,0,0,0,0,0,1,0,0])    #7 pattern
    mse += mlp2.bp([1,1,1,1,1,1,1],[0,0,0,0,0,0,0,0,1,0])    #8 pattern
    mse += mlp2.bp([1,1,1,1,0,1,1],[0,0,0,0,0,0,0,0,0,1])    #9 pattern
    mse = mse/10.0

# Dataset for the 7 to 7 network
print("Training 7 to 7 network...")
for i in range(epochs):
    mse = 0.0
    mse += mlp3.bp([1,1,1,1,1,1,0],[1,1,1,1,1,1,0])    #0 pattern
    mse += mlp3.bp([0,1,1,0,0,0,0],[0,1,1,0,0,0,0])    #1 pattern
    mse += mlp3.bp([1,1,0,1,1,0,1],[1,1,0,1,1,0,1])    #2 pattern
    mse += mlp3.bp([1,1,1,1,0,0,1],[1,1,1,1,0,0,1])    #3 pattern
    mse += mlp3.bp([0,1,1,0,0,1,1],[0,1,1,0,0,1,1])    #4 pattern
    mse += mlp3.bp([1,0,1,1,0,1,1],[1,0,1,1,0,1,1])    #5 pattern
    mse += mlp3.bp([1,0,1,1,1,1,1],[1,0,1,1,1,1,1])    #6 pattern
    mse += mlp3.bp([1,1,1,0,0,0,0],[1,1,1,0,0,0,0])    #7 pattern
    mse += mlp3.bp([1,1,1,1,1,1,1],[1,1,1,1,1,1,1])    #8 pattern
    mse += mlp3.bp([1,1,1,1,0,1,1],[1,1,1,1,0,1,1])    #9 pattern
    mse = mse/10.0

print("Done!\n")
pattern = [1.2]
while(pattern[0]>=0.0):
    pattern = list(map(float, input("Input pattern 'a b c d e f g': ").strip().split()))
    if pattern[0]<0.0:
        break
    print()
    print("The number recognized by the 7 to 1 network is", \
        int(mlp1.run(pattern)*10))

    print("The number recognized by the 7 to 10 network is", \
        np.argmax(mlp2.run(pattern)))

    print("The pattern recognized by the 7 to 7 network is", \

        [int(x) for x in (mlp3.run(pattern) + 0.5)] ,"\n")


Training 7 to 1 network...
Training 7 to 10 network...
Training 7 to 7 network...
Done!



/tmp/ipykernel_56072/2876165150.py:182: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  int(mlp1.run(pattern)*10))



The number recognized by the 7 to 1 network is 1
The number recognized by the 7 to 10 network is 1
The pattern recognized by the 7 to 7 network is [0, 1, 1, 0, 0, 0, 0] 



IndexError: list index out of range

In [8]:
import MLP
from MLP import MultiLayerPerceptron
import tkinter as tk
import numpy as np

sdrnn = MultiLayerPerceptron(layers=[7,7,1])  # MLP
tepochs = 0

def update_a(event):
    r = int(slider_a.get() * (255 - offset)) + offset
    g = offset - int(slider_a.get() * (offset))
    b = offset - int(slider_a.get() * (offset))
    slider_a.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_b(event):
    r = int(slider_b.get() * (255 - offset)) + offset
    g = offset - int(slider_b.get() * (offset))
    b = offset - int(slider_b.get() * (offset))
    slider_b.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()


def update_c(event):
    r = int(slider_c.get() * (255 - offset)) + offset
    g = offset - int(slider_c.get() * (offset))
    b = offset - int(slider_c.get() * (offset))
    slider_c.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()


def update_d(event):
    r = int(slider_d.get() * (255 - offset)) + offset
    g = offset - int(slider_d.get() * (offset))
    b = offset - int(slider_d.get() * (offset))
    slider_d.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()


def update_e(event):
    r = int(slider_e.get() * (255 - offset)) + offset
    g = offset - int(slider_e.get() * (offset))
    b = offset - int(slider_e.get() * (offset))
    slider_e.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()


def update_f(event):
    r = int(slider_f.get() * (255 - offset)) + offset
    g = offset - int(slider_f.get() * (offset))
    b = offset - int(slider_f.get() * (offset))
    slider_f.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()


def update_g(event):
    r = int(slider_g.get() * (255 - offset)) + offset
    g = offset - int(slider_g.get() * (offset))
    b = offset - int(slider_g.get() * (offset))
    slider_g.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def default_btn(event):
    train_callback()

def train_callback():
    global tepochs
    epochs = int(entry_epochs.get())
    for i in range(epochs):
        mse = 0.0
        mse += sdrnn.bp([1,1,1,1,1,1,0],[0.05])    #0 pattern
        mse += sdrnn.bp([0,1,1,0,0,0,0],[0.15])    #1 pattern
        mse += sdrnn.bp([1,1,0,1,1,0,1],[0.25])    #2 pattern
        mse += sdrnn.bp([1,1,1,1,0,0,1],[0.35])    #3 pattern
        mse += sdrnn.bp([0,1,1,0,0,1,1],[0.45])    #4 pattern
        mse += sdrnn.bp([1,0,1,1,0,1,1],[0.55])    #5 pattern
        mse += sdrnn.bp([1,0,1,1,1,1,1],[0.65])    #6 pattern
        mse += sdrnn.bp([1,1,1,0,0,0,0],[0.75])    #7 pattern
        mse += sdrnn.bp([1,1,1,1,1,1,1],[0.85])    #8 pattern
        mse += sdrnn.bp([1,1,1,1,0,1,1],[0.95])    #9 pattern
    lbl_err.configure(text="{0:.10f}".format(mse/10.0))
    tepochs += epochs
    lbl_tepochs.configure(text = tepochs)
    run_ann()
        
def run_ann():
    x = []
    x.append(slider_a.get())
    x.append(slider_b.get())
    x.append(slider_c.get())
    x.append(slider_d.get())
    x.append(slider_e.get())
    x.append(slider_f.get())
    x.append(slider_g.get())    
    theoutput = sdrnn.run(x)[0]
    lbl_out.configure(text="{0:.10f}".format(theoutput))
    lbl_int.configure(text="{}".format(min(int(theoutput*10),9)))

def start_GUI():
    global offset 
    global root
    global slider_a
    global slider_b
    global slider_c
    global slider_d
    global slider_e
    global slider_f
    global slider_g
    global lbl_err
    global lbl_out
    global lbl_int
    global btn_go
    global entry_epochs
    global lbl_tepochs 
    global tepochs 
    tepochs = 0
    offset = 240

    # Create the root object
    root = tk.Tk()
    root.configure(background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))
    root.title("7 to 1 SDRNN")
    root.resizable(0,0)    

    #Create Panedwindow  
    panedwindow = tk.PanedWindow(root, orient='horizontal',
    background="#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))  
    panedwindow.pack(fill="both", expand=True)  
    #Create Frames    
    frame1=tk.Frame(panedwindow,width=100,height=300, relief="sunken", bd=3,
                 background="#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))
    frame2=tk.Frame(panedwindow,width=400,height=400, relief="sunken", bd=3)  

    # Create Scale Widgets
    slider_a = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_a, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_b = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_b, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_c = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_c, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_d = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_d, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_e = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_e, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_f = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_f, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_g = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_g, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))
    
    entry_epochs = tk.Entry(frame2,bd=3, width=10)
    entry_epochs.insert(tk.INSERT, "10")
    entry_epochs.bind('<Return>', default_btn)
    lbl_epochs = tk.Label(frame2, height=2, text="Epochs to Train:")

    lbl_tepochs = tk.Label(frame2, height=2, width=20)
    lbl_tepochs_txt = tk.Label(frame2, height=2, text="Epochs so far:")

    lbl_err = tk.Label(frame2, height=2, width=20)
    lbl_err_txt = tk.Label(frame2, height=2, text="Training Error:")
    
    lbl_out = tk.Label(frame2, height=2, width=20)
    lbl_out_txt = tk.Label(frame2, height=2, text="Raw Output:")
    
    lbl_int = tk.Label(frame2, height=1, width=2, font=("Calibri", 44))
    lbl_int_txt = tk.Label(frame2, height=2, text="Number Output:")
    
    btn_go = tk.Button(frame2, text="Train some more", command = train_callback)
     
    # Use the grid geometry manager to put the widgets in the respective position
    slider_a.grid(row=0, column=1)
    slider_b.grid(row=1, column=2)
    slider_c.grid(row=3, column=2)
    slider_d.grid(row=4, column=1)
    slider_e.grid(row=3, column=0)
    slider_f.grid(row=1, column=0)
    slider_g.grid(row=2, column=1)

    lbl_epochs.grid(      row=0, column=0)
    entry_epochs.grid(    row=0, column=1)
    btn_go.grid(          row=1, column=1)
    lbl_err.grid(         row=2, column=1)
    lbl_err_txt.grid(     row=2, column=0)
    lbl_tepochs_txt.grid( row=3, column=0)
    lbl_tepochs.grid(     row=3, column=1)
    lbl_out.grid(         row=4, column=1)
    lbl_out_txt.grid(     row=4, column=0)
    lbl_int.grid(         row=5, column=1)
    lbl_int_txt.grid(     row=5, column=0)
    
    panedwindow.add(frame1)  
    panedwindow.add(frame2) 
    panedwindow.paneconfig(frame1,minsize=200)
    panedwindow.paneconfig(frame2,minsize=250)

    lbl_err.configure(text="---")
    lbl_tepochs.configure(text = "0")
    run_ann()
    # The application mainloop
    tk.mainloop()

start_GUI()

ModuleNotFoundError: No module named 'MLP'

In [9]:
import Training neural network as MLP
from MLP import MultiLayerPerceptron
import tkinter as tk
import numpy as np

sdrnn = MultiLayerPerceptron(layers=[7,7,7])  # MLP
tepochs = 0

def update_a(event):
    r = int(slider_a.get() * (255 - offset)) + offset
    g = offset - int(slider_a.get() * (offset))
    b = offset - int(slider_a.get() * (offset))
    slider_a.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_b(event):
    r = int(slider_b.get() * (255 - offset)) + offset
    g = offset - int(slider_b.get() * (offset))
    b = offset - int(slider_b.get() * (offset))
    slider_b.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_c(event):
    r = int(slider_c.get() * (255 - offset)) + offset
    g = offset - int(slider_c.get() * (offset))
    b = offset - int(slider_c.get() * (offset))
    slider_c.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_d(event):
    r = int(slider_d.get() * (255 - offset)) + offset
    g = offset - int(slider_d.get() * (offset))
    b = offset - int(slider_d.get() * (offset))
    slider_d.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_e(event):
    r = int(slider_e.get() * (255 - offset)) + offset
    g = offset - int(slider_e.get() * (offset))
    b = offset - int(slider_e.get() * (offset))
    slider_e.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_f(event):
    r = int(slider_f.get() * (255 - offset)) + offset
    g = offset - int(slider_f.get() * (offset))
    b = offset - int(slider_f.get() * (offset))
    slider_f.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_g(event):
    r = int(slider_g.get() * (255 - offset)) + offset
    g = offset - int(slider_g.get() * (offset))
    b = offset - int(slider_g.get() * (offset))
    slider_g.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_out_a(event):
    r = int(out_a.get() * (255 - offset)) + offset
    g = offset - int(out_a.get() * (offset))
    b = offset - int(out_a.get() * (offset))
    out_a.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_b(event):
    r = int(out_b.get() * (255 - offset)) + offset
    g = offset - int(out_b.get() * (offset))
    b = offset - int(out_b.get() * (offset))
    out_b.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_c(event):
    r = int(out_c.get() * (255 - offset)) + offset
    g = offset - int(out_c.get() * (offset))
    b = offset - int(out_c.get() * (offset))
    out_c.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_d(event):
    r = int(out_d.get() * (255 - offset)) + offset
    g = offset - int(out_d.get() * (offset))
    b = offset - int(out_d.get() * (offset))
    out_d.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_e(event):
    r = int(out_e.get() * (255 - offset)) + offset
    g = offset - int(out_e.get() * (offset))
    b = offset - int(out_e.get() * (offset))
    out_e.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_f(event):
    r = int(out_f.get() * (255 - offset)) + offset
    g = offset - int(out_f.get() * (offset))
    b = offset - int(out_f.get() * (offset))
    out_f.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))

def update_out_g(event):
    r = int(out_g.get() * (255 - offset)) + offset
    g = offset - int(out_g.get() * (offset))
    b = offset - int(out_g.get() * (offset))
    out_g.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                 troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
                
def default_btn(event):
    train_callback()

def train_callback():
    global tepochs
    epochs = int(entry_epochs.get())
    for i in range(epochs):
        mse = 0.0
        mse += sdrnn.bp([1,1,1,1,1,1,0],[1,1,1,1,1,1,0])    #0 pattern
        mse += sdrnn.bp([0,1,1,0,0,0,0],[0,1,1,0,0,0,0])    #1 pattern
        mse += sdrnn.bp([1,1,0,1,1,0,1],[1,1,0,1,1,0,1])    #2 pattern
        mse += sdrnn.bp([1,1,1,1,0,0,1],[1,1,1,1,0,0,1])    #3 pattern
        mse += sdrnn.bp([0,1,1,0,0,1,1],[0,1,1,0,0,1,1])    #4 pattern
        mse += sdrnn.bp([1,0,1,1,0,1,1],[1,0,1,1,0,1,1])    #5 pattern
        mse += sdrnn.bp([1,0,1,1,1,1,1],[1,0,1,1,1,1,1])    #6 pattern
        mse += sdrnn.bp([1,1,1,0,0,0,0],[1,1,1,0,0,0,0])    #7 pattern
        mse += sdrnn.bp([1,1,1,1,1,1,1],[1,1,1,1,1,1,1])    #8 pattern
        mse += sdrnn.bp([1,1,1,1,0,1,1],[1,1,1,1,0,1,1])    #9 pattern
    lbl_err.configure(text="{0:.10f}".format(mse/10.0))
    tepochs += epochs
    lbl_tepochs.configure(text = tepochs)
    run_ann()

def run_ann():
    x = []
    x.append(slider_a.get())
    x.append(slider_b.get())
    x.append(slider_c.get())
    x.append(slider_d.get())
    x.append(slider_e.get())
    x.append(slider_f.get())
    x.append(slider_g.get())    
    theoutput = sdrnn.run(x)
    out_a.set(theoutput[0])
    out_b.set(theoutput[1])
    out_c.set(theoutput[2])
    out_d.set(theoutput[3])
    out_e.set(theoutput[4])
    out_f.set(theoutput[5])
    out_g.set(theoutput[6])

def start_GUI():
    global offset 
    global root
    global slider_a
    global slider_b
    global slider_c
    global slider_d
    global slider_e
    global slider_f
    global slider_g
    global lbl_err
    global out_a
    global out_b
    global out_c
    global out_d
    global out_e
    global out_f
    global out_g
    global btn_go
    global entry_epochs
    global lbl_tepochs 
    global tepochs
    tepochs = 0
    offset = 240

    # Create the root object
    root = tk.Tk()
    root.configure(background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))
    root.title("7 to 7 SDRNN")
    root.resizable(0,0)
    
    #Create Panedwindow  
    panedwindow = tk.PanedWindow(root, orient='horizontal',
    background="#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))  
    panedwindow.pack(fill="both", expand=True)  
    #Create Frames    
    frame1=tk.Frame(panedwindow,width=100,height=300, relief="sunken", bd=3,
                 background="#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))
    frame2=tk.Frame(panedwindow,width=400,height=400, relief="sunken", bd=3)  
    frame3=tk.Frame(panedwindow,width=100,height=300, relief="sunken", bd=3,
                 background="#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    # Create Scale Widgets
    slider_a = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_a, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_b = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_b, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_c = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_c, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_d = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_d, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_e = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_e, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_f = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_f, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_g = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_g, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    # Create Scale Widgets
    out_a = tk.Scale(frame3, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_out_a, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_b = tk.Scale(frame3, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_out_b, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_c = tk.Scale(frame3, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_out_c, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_d = tk.Scale(frame3, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_out_d, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_e = tk.Scale(frame3, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_out_e, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_f = tk.Scale(frame3, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_out_f, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    out_g = tk.Scale(frame3, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_out_g, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))
       
    entry_epochs = tk.Entry(frame2, bd=3, width=10)
    entry_epochs.insert(tk.INSERT, "10")
    entry_epochs.bind('<Return>', default_btn)
    lbl_epochs = tk.Label(frame2, height=2, text="Epochs to Train:")

    lbl_tepochs = tk.Label(frame2, height=2, width=20)
    lbl_tepochs_txt = tk.Label(frame2, height=2, text="Epochs so far:")

    lbl_err = tk.Label(frame2, height=2, width=20)
    lbl_err_txt = tk.Label(frame2, height=2, text="Training Error:")
    
    btn_go = tk.Button(frame2, text="Train some more", command = train_callback)
     
    # Use the grid geometry manager to put the widgets in the respective position
    slider_a.grid(row=0, column=1)
    slider_b.grid(row=1, column=2)
    slider_c.grid(row=3, column=2)
    slider_d.grid(row=4, column=1)
    slider_e.grid(row=3, column=0)
    slider_f.grid(row=1, column=0)
    slider_g.grid(row=2, column=1)

    lbl_epochs.grid(      row=0, column=0)
    entry_epochs.grid(    row=0, column=1)
    btn_go.grid(          row=1, column=1)
    lbl_err.grid(         row=2, column=1)
    lbl_err_txt.grid(     row=2, column=0)
    lbl_tepochs_txt.grid( row=3, column=0)
    lbl_tepochs.grid(     row=3, column=1)

    out_a.grid(row=0, column=1)
    out_b.grid(row=1, column=2)
    out_c.grid(row=3, column=2)
    out_d.grid(row=4, column=1)
    out_e.grid(row=3, column=0)
    out_f.grid(row=1, column=0)
    out_g.grid(row=2, column=1)

    panedwindow.add(frame1)  
    panedwindow.add(frame2) 
    panedwindow.add(frame3) 
    panedwindow.paneconfig(frame1,minsize=200)
    panedwindow.paneconfig(frame2,minsize=250)
    panedwindow.paneconfig(frame3,minsize=200)

    lbl_err.configure(text="---")
    lbl_tepochs.configure(text = "0")
    run_ann()
    # The application mainloop
    tk.mainloop()

start_GUI()

SyntaxError: invalid syntax (2234350766.py, line 1)

In [ ]:
import MLP
from MLP import MultiLayerPerceptron
import tkinter as tk
import numpy as np
 
sdrnn = MultiLayerPerceptron(layers=[7,7,10])  # MLP
tepochs = 0

def update_a(event):
    r = int(slider_a.get() * (255 - offset)) + offset
    g = offset - int(slider_a.get() * (offset))
    b = offset - int(slider_a.get() * (offset))
    slider_a.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_b(event):
    r = int(slider_b.get() * (255 - offset)) + offset
    g = offset - int(slider_b.get() * (offset))
    b = offset - int(slider_b.get() * (offset))
    slider_b.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_c(event):
    r = int(slider_c.get() * (255 - offset)) + offset
    g = offset - int(slider_c.get() * (offset))
    b = offset - int(slider_c.get() * (offset))
    slider_c.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_d(event):
    r = int(slider_d.get() * (255 - offset)) + offset
    g = offset - int(slider_d.get() * (offset))
    b = offset - int(slider_d.get() * (offset))
    slider_d.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_e(event):
    r = int(slider_e.get() * (255 - offset)) + offset
    g = offset - int(slider_e.get() * (offset))
    b = offset - int(slider_e.get() * (offset))
    slider_e.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_f(event):
    r = int(slider_f.get() * (255 - offset)) + offset
    g = offset - int(slider_f.get() * (offset))
    b = offset - int(slider_f.get() * (offset))
    slider_f.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def update_g(event):
    r = int(slider_g.get() * (255 - offset)) + offset
    g = offset - int(slider_g.get() * (offset))
    b = offset - int(slider_g.get() * (offset))
    slider_g.config(background="#{0:02x}{1:02x}{2:02x}".format(r,g,b),
                    troughcolor="#{0:02x}{1:02x}{2:02x}".format(r,g,b))
    run_ann()

def default_btn(event):
    train_callback()

def train_callback():
    global tepochs
    epochs = int(entry_epochs.get())
    for i in range(epochs):
        mse = 0.0
        mse += sdrnn.bp([1,1,1,1,1,1,0],[1,0,0,0,0,0,0,0,0,0])    #0 pattern
        mse += sdrnn.bp([0,1,1,0,0,0,0],[0,1,0,0,0,0,0,0,0,0])    #1 pattern
        mse += sdrnn.bp([1,1,0,1,1,0,1],[0,0,1,0,0,0,0,0,0,0])    #2 pattern
        mse += sdrnn.bp([1,1,1,1,0,0,1],[0,0,0,1,0,0,0,0,0,0])    #3 pattern
        mse += sdrnn.bp([0,1,1,0,0,1,1],[0,0,0,0,1,0,0,0,0,0])    #4 pattern
        mse += sdrnn.bp([1,0,1,1,0,1,1],[0,0,0,0,0,1,0,0,0,0])    #5 pattern
        mse += sdrnn.bp([1,0,1,1,1,1,1],[0,0,0,0,0,0,1,0,0,0])    #6 pattern
        mse += sdrnn.bp([1,1,1,0,0,0,0],[0,0,0,0,0,0,0,1,0,0])    #7 pattern
        mse += sdrnn.bp([1,1,1,1,1,1,1],[0,0,0,0,0,0,0,0,1,0])    #8 pattern
        mse += sdrnn.bp([1,1,1,1,0,1,1],[0,0,0,0,0,0,0,0,0,1])    #9 pattern
    lbl_err.configure(text="{0:.10f}".format(mse/10.0))
    tepochs += epochs
    lbl_tepochs.configure(text = tepochs)
    run_ann()        

def run_ann():
    x = []
    x.append(slider_a.get())
    x.append(slider_b.get())
    x.append(slider_c.get())
    x.append(slider_d.get())
    x.append(slider_e.get())
    x.append(slider_f.get())
    x.append(slider_g.get())    
    theoutput = sdrnn.run(x)
    lbl_0.configure(text="{0:.10f}".format(theoutput[0]))
    lbl_1.configure(text="{0:.10f}".format(theoutput[1]))
    lbl_2.configure(text="{0:.10f}".format(theoutput[2]))
    lbl_3.configure(text="{0:.10f}".format(theoutput[3]))
    lbl_4.configure(text="{0:.10f}".format(theoutput[4]))
    lbl_5.configure(text="{0:.10f}".format(theoutput[5]))
    lbl_6.configure(text="{0:.10f}".format(theoutput[6]))
    lbl_7.configure(text="{0:.10f}".format(theoutput[7]))
    lbl_8.configure(text="{0:.10f}".format(theoutput[8]))
    lbl_9.configure(text="{0:.10f}".format(theoutput[9]))
    lbl_int.configure(text="{}".format(np.argmax(theoutput)))

def start_GUI():
    global offset 
    global root
    global slider_a
    global slider_b
    global slider_c
    global slider_d
    global slider_e
    global slider_f
    global slider_g
    global lbl_err
    global lbl_0
    global lbl_1
    global lbl_2
    global lbl_3
    global lbl_4
    global lbl_5
    global lbl_6
    global lbl_7
    global lbl_8
    global lbl_9
    global lbl_int
    global btn_go
    global entry_epochs
    global lbl_tepochs 
    global tepochs
    tepochs = 0
    offset = 240

    # Create the root object
    root = tk.Tk()
    root.configure(background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))
    root.title("7 to 10 SDRNN")
    root.resizable(0,0)

    #Create Panedwindow  
    panedwindow = tk.PanedWindow(root, orient='horizontal',
    background="#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))  
    panedwindow.pack(fill="both", expand=True)  
    #Create Frames
    frame1=tk.Frame(panedwindow,width=100,height=300, relief="sunken", bd=3,
                 background="#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))
    frame2=tk.Frame(panedwindow,width=400,height=400, relief="sunken", bd=3)  

    # Create Scale Widgets
    slider_a = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_a, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_b = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_b, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_c = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_c, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_d = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_d, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_e = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_e, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_f = tk.Scale(frame1, orient="vertical", resolution=0.01,from_=1, to=0,
                            command=update_f, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    slider_g = tk.Scale(frame1, orient="horizontal", resolution=0.01,from_=0, to=1,
                            command=update_g, fg="black",width=10, 
                            background = "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset),
                            troughcolor= "#{0:02x}{1:02x}{2:02x}".format(offset,offset,offset))

    entry_epochs = tk.Entry(frame2, bd=3, width=10)
    entry_epochs.insert(tk.INSERT, "10")
    entry_epochs.bind('<Return>', default_btn)
    lbl_epochs = tk.Label(frame2, height=2, text="Epochs to Train:")

    lbl_tepochs = tk.Label(frame2, height=2, width=20)
    lbl_tepochs_txt = tk.Label(frame2, height=2, text="Epochs so far:")

    lbl_err = tk.Label(frame2, height=2, width=20)
    lbl_err_txt = tk.Label(frame2, height=2, text="Training Error:")
    
    lbl_0 = tk.Label(frame2, height=1, width=20)
    lbl_0_txt = tk.Label(frame2, height=1, text="0 Output:")
    lbl_1 = tk.Label(frame2, height=1, width=20)
    lbl_1_txt = tk.Label(frame2, height=1, text="1 Output:")
    lbl_2 = tk.Label(frame2, height=1, width=20)
    lbl_2_txt = tk.Label(frame2, height=1, text="2 Output:")
    lbl_3 = tk.Label(frame2, height=1, width=20)
    lbl_3_txt = tk.Label(frame2, height=1, text="3 Output:")
    lbl_4 = tk.Label(frame2, height=1, width=20)
    lbl_4_txt = tk.Label(frame2, height=1, text="4 Output:")
    lbl_5 = tk.Label(frame2, height=1, width=20)
    lbl_5_txt = tk.Label(frame2, height=1, text="5 Output:")
    lbl_6 = tk.Label(frame2, height=1, width=20)
    lbl_6_txt = tk.Label(frame2, height=1, text="6 Output:")
    lbl_7 = tk.Label(frame2, height=1, width=20)
    lbl_7_txt = tk.Label(frame2, height=1, text="7 Output:")
    lbl_8 = tk.Label(frame2, height=1, width=20)
    lbl_8_txt = tk.Label(frame2, height=1, text="8 Output:")
    lbl_9 = tk.Label(frame2, height=1, width=20)
    lbl_9_txt = tk.Label(frame2, height=1, text="9 Output:")
    
    lbl_int = tk.Label(frame2, height=1, width=2, font=("Calibri", 44))
    lbl_int_txt = tk.Label(frame2, height=2, text="Number Output:")
    
    btn_go = tk.Button(frame2, text="Train some more", command = train_callback)
     
    # Use the grid geometry manager to put the widgets in the respective position
    slider_a.grid(row=0, column=1)
    slider_b.grid(row=1, column=2)
    slider_c.grid(row=3, column=2)
    slider_d.grid(row=4, column=1)
    slider_e.grid(row=3, column=0)
    slider_f.grid(row=1, column=0)
    slider_g.grid(row=2, column=1)

    lbl_epochs.grid(      row=0, column=0)
    entry_epochs.grid(    row=0, column=1)
    btn_go.grid(          row=1, column=1)
    lbl_err.grid(         row=2, column=1)
    lbl_err_txt.grid(     row=2, column=0)
    lbl_tepochs_txt.grid( row=3, column=0)
    lbl_tepochs.grid(     row=3, column=1)
    lbl_0.grid(           row=4, column=1)
    lbl_0_txt.grid(       row=4, column=0)
    lbl_1.grid(           row=5, column=1)
    lbl_1_txt.grid(       row=5, column=0)
    lbl_2.grid(           row=6, column=1)
    lbl_2_txt.grid(       row=6, column=0)
    lbl_3.grid(           row=7, column=1)
    lbl_3_txt.grid(       row=7, column=0)
    lbl_4.grid(           row=8, column=1)
    lbl_4_txt.grid(       row=8, column=0)
    lbl_5.grid(           row=9, column=1)
    lbl_5_txt.grid(       row=9, column=0)
    lbl_6.grid(           row=10, column=1)
    lbl_6_txt.grid(       row=10, column=0)
    lbl_7.grid(           row=11, column=1)
    lbl_7_txt.grid(       row=11, column=0)
    lbl_8.grid(           row=12, column=1)
    lbl_8_txt.grid(       row=12, column=0)
    lbl_9.grid(           row=13, column=1)
    lbl_9_txt.grid(       row=13, column=0)
    lbl_int.grid(         row=14, column=1)
    lbl_int_txt.grid(     row=14, column=0)
    
    panedwindow.add(frame1)  
    panedwindow.add(frame2) 
    panedwindow.paneconfig(frame1,minsize=200)
    panedwindow.paneconfig(frame2,minsize=250)

    lbl_err.configure(text="---")
    lbl_tepochs.configure(text = "0")
    run_ann()
    # The application mainloop
    tk.mainloop()

start_GUI()